### Imports

In [2]:
# Standard library imports
import os
import re
import unicodedata
import warnings

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import textstat
from textblob import TextBlob
from textblob_nl import PatternAnalyzer
from tqdm.notebook import tqdm
import requests


# 1. Consumentenvertrouwen, economisch klimaat en koopbereidheid

https://opendata.cbs.nl/#/CBS/nl/dataset/83694NED/table?searchKeywords=%E2%80%93%20consumentenvertrouwen,%20economisch%20klimaat%20en%20koopbereidheid

1.1 Data inladen/inzien



De indicatoren kunnen een waarde aannemen van -100 (iedereen antwoordt negatief) tot +100 (iedereen antwoordt positief). Bij een waarde van 0 is het aandeel pessimisten gelijk aan het aandeel optimisten.


In [2]:
import pandas as pd

# 1. Data inladen
df_raw = pd.read_csv(
    r"E:\Nieuw dataset\Consumentenvertrouwen, economisch klimaat en koopbereidheid ongecorrigeerd\Consumentenvertrouwen__economisch_klimaat_en_koopbereidheid__gecorrigeerd_27042025_161526.csv",
    sep=";"
)

display(df_raw)


,Onderwerp,Unnamed: 1,1986 april,1986 mei,1986 juni,1986 juli,1986 augustus,1986 september,1986 oktober,1986 november,...,2024 juli,2024 augustus,2024 september,2024 oktober,2024 november,2024 december,2025 januari,2025 februari,2025 maart,2025 april
0,Consumentenvertrouwen,NaN,2,8,15,20,21,21,21,21,...,-24,-24,-21,-22,-25,-26,-28,-32,-34,-37
1,Economisch klimaat,NaN,6,17,33,45,47,45,45,44,...,-39,-42,-37,-37,-43,-45,-50,-54,-57,-61
2,Koopbereidheid,NaN,0,1,3,4,4,5,6,5,...,-14,-13,-11,-12,-12,-13,-14,-17,-18,-20


1.2 Data opschonen 
- data transposen

In [3]:


# 2. Transponeren
df_transposed = df_raw.set_index('Onderwerp').T

# 3. Index hernoemen naar 'Periode'
df_transposed.index.name = 'Periode'
df_transposed.reset_index(inplace=True)

# 4. Displayen
display(df_transposed)


Onderwerp,Periode,Consumentenvertrouwen,Economisch klimaat,Koopbereidheid
0,Unnamed: 1,NaN,NaN,NaN
1,1986 april,2.0,6.0,0.0
2,1986 mei,8.0,17.0,1.0
3,1986 juni,15.0,33.0,3.0
4,1986 juli,20.0,45.0,4.0
...,...,...,...,...
465,2024 december,-26.0,-45.0,-13.0
466,2025 januari,-28.0,-50.0,-14.0
467,2025 februari,-32.0,-54.0,-17.0
468,2025 maart,-34.0,-57.0,-18.0


-  perioden wijzigen naar yy/mm 
- datum selecteren 2015-1 t/m 2025-01

In [4]:


# 5. Periode omzetten naar datetime
df_transposed['Periode'] = pd.to_datetime(df_transposed['Periode'], format='%Y %B', errors='coerce')

# 6. Omzetten naar YYYY/MM string
df_transposed['Periode'] = df_transposed['Periode'].dt.strftime('%Y/%m')

# 7. Filteren op periode 2015-01 t/m 2025-04
df_transposed = df_transposed[
    (df_transposed['Periode'] >= '2015/01') & 
    (df_transposed['Periode'] <= '2025/04')
]

# 8. Kolommen herschikken (alleen als ze bestaan)
kolommen = ['Periode']
for kolom in ['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']:
    if kolom in df_transposed.columns:
        kolommen.append(kolom)

df_transposed = df_transposed[kolommen]

# 9. Resultaat bekijken
display(df_transposed)

Onderwerp,Periode,Consumentenvertrouwen,Economisch klimaat,Koopbereidheid
349,2015/04,10.0,30.0,-4.0
354,2015/09,11.0,28.0,0.0
356,2015/11,14.0,31.0,3.0
357,2015/12,13.0,27.0,4.0
361,2016/04,6.0,12.0,1.0
366,2016/09,12.0,26.0,3.0
368,2016/11,20.0,39.0,8.0
369,2016/12,21.0,39.0,8.0
373,2017/04,26.0,50.0,10.0
378,2017/09,23.0,44.0,10.0


Data wordt weergeven in kwartalen, ga ervoor kiezen om Interpolatie toe te voegen 

In [5]:
# ---- Optie 2: Interpolatie toevoegen ----

# Genereer volledige maandlijst
maanden_volledig = pd.date_range(start='2015-01-01', end='2025-04-01', freq='MS').strftime('%Y/%m')
df_maanden = pd.DataFrame({'Periode': maanden_volledig})

# Merge volledige maanden met de transposed data
df_consumentenvertrouwen_interpolatie = df_maanden.merge(df_transposed, on='Periode', how='left')

# Interpoleren (lineair) op de relevante kolommen
df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']] = \
    df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']].interpolate(method='linear')

# 1. Eerst interpoleren
df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']] = \
    df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']].interpolate(method='linear')

# 2. Daarna backward fill voor beginwaarden
df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']] = \
    df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']].fillna(method='bfill')

# Resultaat bekijken
display(df_consumentenvertrouwen_interpolatie)

C:\Users\dylan\AppData\Local\Temp\ipykernel_9080\1763514700.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']].fillna(method='bfill')


,Periode,Consumentenvertrouwen,Economisch klimaat,Koopbereidheid
0,2015/01,10.00,30.0,-4.00
1,2015/02,10.00,30.0,-4.00
2,2015/03,10.00,30.0,-4.00
3,2015/04,10.00,30.0,-4.00
4,2015/05,10.20,29.6,-3.20
...,...,...,...,...
119,2024/12,-26.00,-45.0,-13.00
120,2025/01,-28.75,-49.0,-14.75
121,2025/02,-31.50,-53.0,-16.50
122,2025/03,-34.25,-57.0,-18.25


1.3 Data exporteren naar csv

In [ ]:
# Resultaat bekijken
display(df_consumentenvertrouwen_interpolatie)

# Opslaan als CSV
output_path = r"E:\Nieuw dataset\Consumentenvertrouwen, economisch klimaat en koopbereidheid ongecorrigeerd\consumentenvertrouwen_interpolatie.csv"
df_consumentenvertrouwen_interpolatie.to_csv(output_path, index=False)

# Pad van de nieuwe CSV printen
print(f"CSV opgeslagen op: {output_path}")

,Periode,Consumentenvertrouwen,Economisch klimaat,Koopbereidheid
0,2015/01,10.00,30.0,-4.00
1,2015/02,10.00,30.0,-4.00
2,2015/03,10.00,30.0,-4.00
3,2015/04,10.00,30.0,-4.00
4,2015/05,10.20,29.6,-3.20
...,...,...,...,...
119,2024/12,-26.00,-45.0,-13.00
120,2025/01,-28.75,-49.0,-14.75
121,2025/02,-31.50,-53.0,-16.50
122,2025/03,-34.25,-57.0,-18.25


CSV opgeslagen op: E:\Nieuw dataset\Consumentenvertrouwen, economisch klimaat en koopbereidheid ongecorrigeerd\consumentenvertrouwen_interpolatie.csv


# 2. Pompprijzen motorbrandstoffen

https://opendata.cbs.nl/#/CBS/nl/dataset/80416ned/table?ts=1745763454832 

2.1 Data inladen/inzien

In [52]:
# Dataset 2 inladen
df_pomprijzen = pd.read_csv(
    r"E:\Nieuw dataset\CBS POMPPRIJZEN\CBS Bnezine prijzen.csv",
    sep=";"
)
display(df_pomprijzen)


,Perioden,BenzineEuro95_1,Diesel_2
0,20060101,1.325,1.003
1,20060102,1.328,1.007
2,20060103,1.332,1.007
3,20060104,1.348,1.020
4,20060105,1.347,1.021
...,...,...,...
7032,20250403,1.945,1.683
7033,20250404,1.938,1.674
7034,20250405,1.927,1.660
7035,20250406,1.927,1.659


2.2 Data opschonen 

-  perioden wijzigen naar yy/mm 
- datum selecteren 2015-1 t/m 2025-01

In [53]:
# Perioden naar datetime
# Datumkolom omzetten (bijvoorbeeld 'Perioden')
df_pomprijzen['Perioden'] = pd.to_datetime(df_pomprijzen['Perioden'], format='%Y%m%d', errors='coerce')

df_pomprijzen['Perioden'] = pd.to_datetime(df_pomprijzen['Perioden'], format='%Y %B')

# Filter op periode
df_pomprijzen = df_pomprijzen[
    (df_pomprijzen['Perioden'] >= '2015-01-01') &
    (df_pomprijzen['Perioden'] <= '2025-01-31')
]

# Eventueel: per maand aggregeren (gemiddelde benzineprijzen bijvoorbeeld)
# df_pomprijzen.groupby(df_pomprijzen['Perioden'].dt.to_period('M')).mean()

display(df_pomprijzen)


,Perioden,BenzineEuro95_1,Diesel_2
3287,2015-01-01,1.501,1.207
3288,2015-01-02,1.503,1.208
3289,2015-01-03,1.498,1.206
3290,2015-01-04,1.500,1.206
3291,2015-01-05,1.501,1.208
...,...,...,...
6966,2025-01-27,1.969,1.767
6967,2025-01-28,1.963,1.760
6968,2025-01-29,1.960,1.754
6969,2025-01-30,1.962,1.753


- Perioden omzetten naar per maand (first, last, min, mean, avg voor benzine en diesel)

In [54]:
# 1. Periode omzetten naar YYYY/MM (zonder tijd)
df_pomprijzen['Periode'] = df_pomprijzen['Perioden'].dt.to_period('M').astype(str)

# 2. Groeperen per maand en aggregaties toepassen
df_pomprijzen_aggregatie = df_pomprijzen.groupby('Periode').agg({
    'BenzineEuro95_1': ['first', 'last', 'min', 'max', 'mean'],
    'Diesel_2': ['first', 'last', 'min', 'max', 'mean']
}).reset_index()

# 3. Kolomnamen netter maken (flatten MultiIndex)
df_pomprijzen_aggregatie.columns = ['Periode'] + [
    f'{brandstof}_{stat}' for brandstof, stat in df_pomprijzen_aggregatie.columns.tolist()[1:]
]

# 4. Resultaat tonen
display(df_pomprijzen_aggregatie.head(15))


,Periode,BenzineEuro95_1_first,BenzineEuro95_1_last,BenzineEuro95_1_min,BenzineEuro95_1_max,BenzineEuro95_1_mean,Diesel_2_first,Diesel_2_last,Diesel_2_min,Diesel_2_max,Diesel_2_mean
0,2015-01,1.501,1.460,1.458,1.503,1.473065,1.207,1.190,1.184,1.208,1.194065
1,2015-02,1.461,1.557,1.461,1.557,1.520143,1.192,1.291,1.192,1.291,1.253964
2,2015-03,1.559,1.601,1.559,1.603,1.586161,1.291,1.277,1.276,1.294,1.286613
3,2015-04,1.599,1.646,1.592,1.647,1.614467,1.275,1.312,1.261,1.312,1.283367
4,2015-05,1.645,1.639,1.634,1.648,1.640806,1.310,1.306,1.306,1.313,1.309710
5,2015-06,1.638,1.653,1.638,1.668,1.654933,1.304,1.295,1.293,1.310,1.301400
6,2015-07,1.654,1.628,1.627,1.685,1.659806,1.295,1.220,1.220,1.295,1.264516
7,2015-08,1.628,1.521,1.520,1.632,1.584258,1.218,1.157,1.155,1.220,1.190742
8,2015-09,1.521,1.487,1.487,1.521,1.510733,1.165,1.182,1.165,1.201,1.190633
9,2015-10,1.484,1.476,1.466,1.497,1.481452,1.181,1.173,1.169,1.201,1.183613


In [55]:
# 1. Periode omzetten naar YYYY/MM (zonder tijd)
df_pomprijzen['Periode'] = df_pomprijzen['Perioden'].dt.to_period('M').astype(str)

# 2. Groeperen per maand en aggregaties toepassen
df_pomprijzen_aggregatie = df_pomprijzen.groupby('Periode').agg({
    'BenzineEuro95_1': ['first', 'last', 'min', 'max', 'mean'],
    'Diesel_2': ['first', 'last', 'min', 'max', 'mean']
}).reset_index()

# Na df_pomprijzen_aggregatie aanmaken:
df_pomprijzen_aggregatie['Periode'] = df_pomprijzen_aggregatie['Periode'].str.replace('-', '/')


# 3. Kolomnamen netter maken (flatten MultiIndex)
df_pomprijzen_aggregatie.columns = ['Periode'] + [
    f'{brandstof}_{stat}' for brandstof, stat in df_pomprijzen_aggregatie.columns.tolist()[1:]
]

# 4. Resultaat tonen
display(df_pomprijzen_aggregatie)


,Periode,BenzineEuro95_1_first,BenzineEuro95_1_last,BenzineEuro95_1_min,BenzineEuro95_1_max,BenzineEuro95_1_mean,Diesel_2_first,Diesel_2_last,Diesel_2_min,Diesel_2_max,Diesel_2_mean
0,2015/01,1.501,1.460,1.458,1.503,1.473065,1.207,1.190,1.184,1.208,1.194065
1,2015/02,1.461,1.557,1.461,1.557,1.520143,1.192,1.291,1.192,1.291,1.253964
2,2015/03,1.559,1.601,1.559,1.603,1.586161,1.291,1.277,1.276,1.294,1.286613
3,2015/04,1.599,1.646,1.592,1.647,1.614467,1.275,1.312,1.261,1.312,1.283367
4,2015/05,1.645,1.639,1.634,1.648,1.640806,1.310,1.306,1.306,1.313,1.309710
...,...,...,...,...,...,...,...,...,...,...,...
116,2024/09,1.912,1.861,1.859,1.916,1.879633,1.635,1.589,1.584,1.640,1.607667
117,2024/10,1.860,1.914,1.859,1.926,1.902806,1.588,1.649,1.587,1.660,1.640419
118,2024/11,1.914,1.955,1.912,1.986,1.947800,1.652,1.744,1.648,1.774,1.716167
119,2024/12,1.953,1.942,1.940,1.958,1.947871,1.740,1.720,1.716,1.748,1.725774


2.3 CSV exporteren

In [56]:
# Opslaan als CSV
output_path = r"E:\Nieuw dataset\CBS POMPPRIJZEN\df_pomprijzen_aggregatie.csv"
df_pomprijzen_aggregatie.to_csv(output_path, index=False)

# Pad van de nieuwe CSV printen
print(f"CSV opgeslagen op: {output_path}")

CSV opgeslagen op: E:\Nieuw dataset\CBS POMPPRIJZEN\df_pomprijzen_aggregatie.csv


# 3. Verkochte wegvoertuigen

https://opendata.cbs.nl/#/CBS/nl/dataset/85898NED/table?ts=1745764299955

3.1 Data inladen/inzien

In [36]:
# Dataset 3 inladen
df_wegvoertuigenRaw = pd.read_csv(
    r"E:\Nieuw dataset\Verkochte wegvoertuigen\Verkochte_wegvoertuigen_Full elektric.csv",
    sep=";"
)

display(df_wegvoertuigenRaw)

,Wegvoertuigen,Brandstofsoort voertuig,Perioden,Totaal verkochte wegvoertuigen (aantal)
0,Totaal wegvoertuigen,Full elektric (BEV),2007 januari,31
1,Totaal wegvoertuigen,Full elektric (BEV),2007 februari,39
2,Totaal wegvoertuigen,Full elektric (BEV),2007 maart,123
3,Totaal wegvoertuigen,Full elektric (BEV),2007 april,62
4,Totaal wegvoertuigen,Full elektric (BEV),2007 mei,84
...,...,...,...,...
214,Totaal wegvoertuigen,Full elektric (BEV),2024 november*,32623
215,Totaal wegvoertuigen,Full elektric (BEV),2024 december*,39594
216,Totaal wegvoertuigen,Full elektric (BEV),2025 januari*,33680
217,Totaal wegvoertuigen,Full elektric (BEV),2025 februari*,29071


3.2 Data opschonen 
- Perioden omzetten naar per maand (first, last, min, mean, avg voor benzine en diesel)
-  perioden wijzigen naar yy/mm 
- datum selecteren 2015-1 t/m 2025-01

In [48]:
# 1. Sterretjes verwijderen
df_wegvoertuigenRaw['Perioden'] = df_wegvoertuigenRaw['Perioden'].str.replace('*', '', regex=False).str.strip()

# 2. Nederlandse maandnamen vervangen door Engelse
for nl, eng in maand_mapping.items():
    df_wegvoertuigenRaw['Perioden'] = df_wegvoertuigenRaw['Perioden'].str.replace(nl, eng, regex=False)

# 3. Periode omzetten naar datetime
df_wegvoertuigenRaw['Periode'] = pd.to_datetime(df_wegvoertuigenRaw['Perioden'], format='%Y %B', errors='coerce')

# 4. Omzetten naar YYYY/MM string
df_wegvoertuigenRaw['Periode'] = df_wegvoertuigenRaw['Periode'].dt.strftime('%Y/%m')

# 5. Alleen relevante kolommen behouden
df_wegvoertuigen = df_wegvoertuigenRaw[['Periode', 'Totaal verkochte wegvoertuigen (aantal)']]

# 5b. Filter op periode 2015/01 t/m 2025/03
df_wegvoertuigen = df_wegvoertuigen[
    (df_wegvoertuigen['Periode'] >= '2015/01') & 
    (df_wegvoertuigen['Periode'] <= '2025/03')
]

# 6. Kolom hernoemen
df_wegvoertuigen.rename(columns={'Totaal verkochte wegvoertuigen (aantal)': 'Totaal_verkochte_wegvoertuigen'}, inplace=True)

# 7. Resultaat bekijken
display(df_wegvoertuigen)


,Periode,Totaal_verkochte_wegvoertuigen
96,2015/01,1202
97,2015/02,926
98,2015/03,1615
99,2015/04,1362
100,2015/05,1332
...,...,...
214,2024/11,32623
215,2024/12,39594
216,2025/01,33680
217,2025/02,29071


Er is iets misgegaan met de data, krijg steeds maar 40 rijen teryg terwijl ik er 121 verwacht, ik heb zelf de benodigde waarden van "E:\Nieuw dataset\Verkochte wegvoertuigen\Verkochte_wegvoertuigen_Full elektric.csv gekopieerd en geplakt in een nieuw csv.

3.3 CSV exporteren

In [50]:
# Opslaan als CSV
output_path = r"E:\Nieuw dataset\Verkochte wegvoertuigen\df_wegvoertuigen_nieuw.csv"
df_wegvoertuigen_nieuw.to_csv(output_path, index=False)

# Pad van de nieuwe CSV printen
print(f"CSV opgeslagen op: {output_path}")

CSV opgeslagen op: E:\Nieuw dataset\Verkochte wegvoertuigen\df_wegvoertuigen_nieuw.csv


# Alles joinen 

In [66]:
# CSV's inladen
df_pomprijzen = pd.read_csv(r"e:\Nieuw dataset\CBS POMPPRIJZEN\df_pomprijzen_aggregatie.csv")
df_consumentenvertrouwen = pd.read_csv(r"E:\Nieuw dataset\Consumentenvertrouwen, economisch klimaat en koopbereidheid ongecorrigeerd\consumentenvertrouwen_interpolatie.csv")
df_wegvoertuigen = pd.read_csv(r"E:\Nieuw dataset\Verkochte wegvoertuigen\df_wegvoertuigen_nieuw.csv")

# Joinen op 'Periode' (left join op consumentenvertrouwen als basis)
df_joined = pd.merge(df_consumentenvertrouwen, df_pomprijzen, on='Periode', how='left')
df_joined = pd.merge(df_joined, df_wegvoertuigen, on='Periode', how='left')
# Filteren op periodes tot en met 2025/01


# Resultaat bekijken
display(df_joined)


,Periode,Consumentenvertrouwen,Economisch klimaat,Koopbereidheid,BenzineEuro95_1_first,BenzineEuro95_1_last,BenzineEuro95_1_min,BenzineEuro95_1_max,BenzineEuro95_1_mean,Diesel_2_first,Diesel_2_last,Diesel_2_min,Diesel_2_max,Diesel_2_mean,Totaal_verkochte_wegvoertuigen
0,2015/01,10.00,30.0,-4.00,1.501,1.460,1.458,1.503,1.473065,1.207,1.190,1.184,1.208,1.194065,1202.0
1,2015/02,10.00,30.0,-4.00,1.461,1.557,1.461,1.557,1.520143,1.192,1.291,1.192,1.291,1.253964,926.0
2,2015/03,10.00,30.0,-4.00,1.559,1.601,1.559,1.603,1.586161,1.291,1.277,1.276,1.294,1.286613,1615.0
3,2015/04,10.00,30.0,-4.00,1.599,1.646,1.592,1.647,1.614467,1.275,1.312,1.261,1.312,1.283367,1362.0
4,2015/05,10.20,29.6,-3.20,1.645,1.639,1.634,1.648,1.640806,1.310,1.306,1.306,1.313,1.309710,1332.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2024/12,-26.00,-45.0,-13.00,1.953,1.942,1.940,1.958,1.947871,1.740,1.720,1.716,1.748,1.725774,39594.0
120,2025/01,-28.75,-49.0,-14.75,1.942,1.960,1.942,1.979,1.960935,1.719,1.752,1.719,1.785,1.753516,33680.0
121,2025/02,-31.50,-53.0,-16.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,2025/03,-34.25,-57.0,-18.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# Missende waarden per kolom tellen
missing_values = df_joined.isnull().sum()

missing_values

Periode                           0
Consumentenvertrouwen             0
Economisch klimaat                0
Koopbereidheid                    0
BenzineEuro95_1_first             3
BenzineEuro95_1_last              3
BenzineEuro95_1_min               3
BenzineEuro95_1_max               3
BenzineEuro95_1_mean              3
Diesel_2_first                    3
Diesel_2_last                     3
Diesel_2_min                      3
Diesel_2_max                      3
Diesel_2_mean                     3
Totaal_verkochte_wegvoertuigen    3
dtype: int64

> data t/m  2025/04  is meegnomen  

In [68]:
df_joined = df_joined[df_joined['Periode'] <= '2025/01']
display(df_joined)

,Periode,Consumentenvertrouwen,Economisch klimaat,Koopbereidheid,BenzineEuro95_1_first,BenzineEuro95_1_last,BenzineEuro95_1_min,BenzineEuro95_1_max,BenzineEuro95_1_mean,Diesel_2_first,Diesel_2_last,Diesel_2_min,Diesel_2_max,Diesel_2_mean,Totaal_verkochte_wegvoertuigen
0,2015/01,10.00,30.0,-4.00,1.501,1.460,1.458,1.503,1.473065,1.207,1.190,1.184,1.208,1.194065,1202.0
1,2015/02,10.00,30.0,-4.00,1.461,1.557,1.461,1.557,1.520143,1.192,1.291,1.192,1.291,1.253964,926.0
2,2015/03,10.00,30.0,-4.00,1.559,1.601,1.559,1.603,1.586161,1.291,1.277,1.276,1.294,1.286613,1615.0
3,2015/04,10.00,30.0,-4.00,1.599,1.646,1.592,1.647,1.614467,1.275,1.312,1.261,1.312,1.283367,1362.0
4,2015/05,10.20,29.6,-3.20,1.645,1.639,1.634,1.648,1.640806,1.310,1.306,1.306,1.313,1.309710,1332.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2024/09,-21.00,-37.0,-11.00,1.912,1.861,1.859,1.916,1.879633,1.635,1.589,1.584,1.640,1.607667,28820.0
117,2024/10,-23.00,-40.0,-11.50,1.860,1.914,1.859,1.926,1.902806,1.588,1.649,1.587,1.660,1.640419,30491.0
118,2024/11,-25.00,-43.0,-12.00,1.914,1.955,1.912,1.986,1.947800,1.652,1.744,1.648,1.774,1.716167,32623.0
119,2024/12,-26.00,-45.0,-13.00,1.953,1.942,1.940,1.958,1.947871,1.740,1.720,1.716,1.748,1.725774,39594.0


In [69]:
# Missende waarden per kolom tellen
missing_values = df_joined.isnull().sum()

missing_values


Periode                           0
Consumentenvertrouwen             0
Economisch klimaat                0
Koopbereidheid                    0
BenzineEuro95_1_first             0
BenzineEuro95_1_last              0
BenzineEuro95_1_min               0
BenzineEuro95_1_max               0
BenzineEuro95_1_mean              0
Diesel_2_first                    0
Diesel_2_last                     0
Diesel_2_min                      0
Diesel_2_max                      0
Diesel_2_mean                     0
Totaal_verkochte_wegvoertuigen    0
dtype: int64

Opslaan als csv

In [70]:
df_joined.to_csv(
    r"E:\Nieuw dataset\df_joined_datasets.csv",
    index=False
)

### nos JOINEN 


In [3]:
# Nieuwe dataset inladen
df_joined = pd.read_csv(r"E:\Nieuw dataset\df_joined_datasets.csv")
df_sentiment = pd.read_csv(
    r"E:\Nieuw dataset\Dutch news articles 2015-2025 NOS.nl\average_sentiment_per_month.csv"
)

# Kolom hernoemen in df_sentiment
df_sentiment.rename(columns={'Year-Month': 'Periode'}, inplace=True)

# Joinen op 'Periode'
df_joined = pd.merge(df_joined, df_sentiment, on='Periode', how='left')

# Zet de kolom 'Average Sentiment Score' op de één-na-laatste positie
cols = df_joined.columns.tolist()

# Verplaats de sentimentkolom naar de juiste plek
sentiment_col = 'Average Sentiment Score'
if sentiment_col in cols:
    cols.remove(sentiment_col)
    cols.insert(-1, sentiment_col)  # zet op één-na-laatste plek

# Herorden de DataFrame
df_joined = df_joined[cols]

# Resultaat bekijken
display(df_joined)




,Periode,Consumentenvertrouwen,Economisch klimaat,Koopbereidheid,BenzineEuro95_1_first,BenzineEuro95_1_last,BenzineEuro95_1_min,BenzineEuro95_1_max,BenzineEuro95_1_mean,Diesel_2_first,Diesel_2_last,Diesel_2_min,Diesel_2_max,Diesel_2_mean,Average Sentiment Score,Totaal_verkochte_wegvoertuigen
0,2015/01,10.00,30.0,-4.00,1.501,1.460,1.458,1.503,1.473065,1.207,1.190,1.184,1.208,1.194065,0.940040,1202.0
1,2015/02,10.00,30.0,-4.00,1.461,1.557,1.461,1.557,1.520143,1.192,1.291,1.192,1.291,1.253964,0.942388,926.0
2,2015/03,10.00,30.0,-4.00,1.559,1.601,1.559,1.603,1.586161,1.291,1.277,1.276,1.294,1.286613,0.931581,1615.0
3,2015/04,10.00,30.0,-4.00,1.599,1.646,1.592,1.647,1.614467,1.275,1.312,1.261,1.312,1.283367,0.949880,1362.0
4,2015/05,10.20,29.6,-3.20,1.645,1.639,1.634,1.648,1.640806,1.310,1.306,1.306,1.313,1.309710,0.926799,1332.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2024/09,-21.00,-37.0,-11.00,1.912,1.861,1.859,1.916,1.879633,1.635,1.589,1.584,1.640,1.607667,0.938640,28820.0
117,2024/10,-23.00,-40.0,-11.50,1.860,1.914,1.859,1.926,1.902806,1.588,1.649,1.587,1.660,1.640419,0.932081,30491.0
118,2024/11,-25.00,-43.0,-12.00,1.914,1.955,1.912,1.986,1.947800,1.652,1.744,1.648,1.774,1.716167,0.924281,32623.0
119,2024/12,-26.00,-45.0,-13.00,1.953,1.942,1.940,1.958,1.947871,1.740,1.720,1.716,1.748,1.725774,0.940189,39594.0


In [4]:
# Opslaan als CSV
df_joined.to_csv(
    r"E:\Nieuw dataset\df_joined_datasets_NOS.csv",
    index=False
)